# VGG Model

In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image
import warnings
import os 

In [25]:
train_dir=r"Data/train"
test_dir=r"Data/test"


In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *

In [27]:
train_data_gen=ImageDataGenerator()
test_data_gen=ImageDataGenerator()
test_data=train_data_gen.flow_from_directory(test_dir, target_size=(224,224),batch_size=32,class_mode="categorical")
train_data=train_data_gen.flow_from_directory(train_dir, target_size=(224,224),batch_size=32,class_mode="categorical")

Found 1288 images belonging to 3 classes.


Found 5144 images belonging to 3 classes.


In [28]:
num_classes=train_data.num_classes
num_classes

3

In [29]:
class_names=train_data.class_indices
print(class_names)

{'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2}


In [30]:
def plot_curves(history):    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping=EarlyStopping(patience=3,monitor="val_loss")


In [32]:
from tensorflow.keras.applications import DenseNet201
densenet_201=DenseNet201(input_shape=(224,224,3),classes=num_classes,include_top=False,weights='imagenet')
input=Input(name="Input_Layer",shape=(224,224,3))
x=tf.keras.layers.Rescaling(1./255)(input)
x=densenet_201(x)
x=GlobalAveragePooling2D()(x)
x=Dense(512, activation='relu')(x)
x=Dropout(0.5)(x)
# x=GlobalAveragePooling2D()(x)
# x=Dropout(0.2)(x)
# x=tensorflow.keras.layers.Flatten()(x)
# x=tensorflow.keras.layers.Dense(512,activation='relu')(x)
output=Dense(num_classes,activation='softmax',name='Output_Layer')(x)
model_densenet_201=tf.keras.Model(input,output)

In [33]:
model_densenet_201.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

In [34]:
model_densenet_201.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet201 (Functional)        │ (None, 7, 7, 1920)     │    18,321,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1920)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       983,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,307,075 (73.65 MB)

 Trainable params: 19,078,019 (72.78 MB)

 Non-trainable params: 229,056 (894.75 KB)

In [ ]:
history_densenet_201=model_densenet_201.fit(train_data,validation_data=test_data,epochs=1,callbacks=[early_stopping])


In [ ]:
plot_curves(history_densenet_201)


In [ ]:
test_loss_densenet, test_acc_densenet = model_densenet_201.evaluate(test_data)


In [ ]:
model_densenet_201.save("Densenet.keras")

In [ ]:
model_densenet_201.predict(test_data)

NameError: name 'model_densenet_201' is not defined